# NeVA Training / Inference Tutorial

### Note:
Currently, this notebook must be run in a NeMo container. An example command to launch the container:

```
docker run --gpus all -it --rm -v <your_nemo_dir>:/opt/NeMo --shm-size=8g \
     -p 8888:8888 --ulimit memlock=-1 --ulimit \
      stack=67108864 <your_nemo_container>
```

## Introduction

This notebook illustrates how to train and perform inference using NeVA with the NeMo Toolkit. NeVA originates from [LLaVA](https://github.com/haotian-liu/LLaVA) (Large Language and Vision Assistant) and is a powerful multimodal image-text instruction tuned model optimized within the NeMo Framework. 


This tutorial will guide you through the following topics:
1. Training a NeVA model
2. Performing inference with the trained model

## Datasets

After downloading all below datasets for pretraining and instruction tuning, your dataset directory should look something similar to:

```
LLaVA-Pretrain-LCS-558K
├── blip_laion_cc_sbu_558k.json
├── images
LLaVA-Instruct-mixture
├── llava_v1_5_mix665k.json
└── images
    └── ...
```

### Pre-Training Dataset

The pre-training dataset is open-sourced from the LLaVA implementation and can be downloaded [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Pretrain). The dataset consists of a 558K subset of the LAION-CC-SBU dataset with BLIP captions. 

The associated images for pretraining can be downloaded via HuggingFace [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Pretrain/blob/main/images.zip).

### Instruction Tuning Dataset

The instruction tuning annotations are sourced from the LLaVA implementation and are available [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/blob/main/llava_v1_5_mix665k.json).

The associated images for the mixture instruction tuning annotations can be found [here](https://github.com/haotian-liu/LLaVA?tab=readme-ov-file#visual-instruction-tuning). After extracting, the data should be formatted as follows:

```
    images
      ├── coco
      │    └── train2017
      ├── gqa
      │    └── images
      ├── ocr_vqa
      │    └── images
      ├── textvqa
      │    └── train_images
      └── vg
           ├── VG_100K
           └── VG_100K_2
```

## Training


### Feature Alignment Pre-Training

We provide a set of scripts for pre-training and fine-tuning which can be kicked off with CLI flags defining specified arguments. 

An example of a pre-training script execution:

In [ ]:
! torchrun --nproc_per_node=4 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_pretrain.py \
 ++cluster_type=BCP \
 trainer.precision=bf16 \
 trainer.num_nodes=1 \
 trainer.devices=4 \
 trainer.val_check_interval=1000 \
 trainer.limit_val_batches=5 \
 trainer.log_every_n_steps=1 \
 trainer.max_steps=1000 \
 model.megatron_amp_O2=True \
 model.micro_batch_size=1 \
 model.global_batch_size=2 \
 model.tensor_model_parallel_size=4 \
 model.pipeline_model_parallel_size=1 \
 model.mcore_gpt=True \
 model.transformer_engine=True \
 model.data.data_path=/path/to/datasets/LLaVA-Pretrain-LCS-558K/blip_laion_cc_sbu_558k.json \
 model.data.image_folder=/path/to/dataset/LLaVA-Pretrain-LCS-558K/images \
 model.tokenizer.library=sentencepiece \
 model.tokenizer.model=/path/to/tokenizer/model \
 model.encoder_seq_length=4096 \
 model.num_layers=32 \
 model.hidden_size=4096 \
 model.ffn_hidden_size=16384 \
 model.num_attention_heads=32 \
 model.normalization=layernorm1p \
 model.do_layer_norm_weight_decay=False \
 model.apply_query_key_layer_scaling=True \
 model.activation=squared-relu \
 model.headscale=False \
 model.position_embedding_type=rope \
 model.rotary_percentage=0.5 \
 model.num_query_groups=null \
 model.data.num_workers=0 \
 model.mm_cfg.llm.from_pretrained=/path/to/checkpoint \
 model.mm_cfg.llm.model_type=nvgpt \
 model.data.conv_template=nvgpt \
 model.mm_cfg.vision_encoder.from_pretrained='openai/clip-vit-large-patch14' \
 model.mm_cfg.vision_encoder.from_hf=True \
 model.data.image_token_len=256 \
 model.optim.name="fused_adam" \
 exp_manager.create_checkpoint_callback=True \
 exp_manager.create_wandb_logger=False \
 exp_manager.wandb_logger_kwargs.project=neva_demo



**Note**: To initialize training a model from scratch rather than from a pretrained checkpoint, you may specify `null` instead of a path in the CLI arguments.

### Image-Language Pair Instruction Fine-Tuning

Fine-tuning can also be run from within the container via a similar command leveraging the `neva_finetune.py` script.

An example of an image-text pair instruction tuning script execution:

In [ ]:
! torchrun --nproc_per_node=4 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_finetune.py \
++cluster_type=BCP \
 trainer.precision=bf16 \
 trainer.num_nodes=1 \
 trainer.devices=1 \
 trainer.val_check_interval=100 \
 trainer.limit_val_batches=50 \
 trainer.max_steps=4900 \
 model.megatron_amp_O2=True \
 model.micro_batch_size=4 \
 model.global_batch_size=32 \
 model.tensor_model_parallel_size=1 \
 model.pipeline_model_parallel_size=1 \
 model.mcore_gpt=True \
 model.transformer_engine=True \
 model.data.data_path=/path/to/dataset/LLaVA-Pretrain-LCS-558K/blip_laion_cc_sbu_558k.json \
 model.data.image_folder=/path/to/dataset/LLaVA-Pretrain-LCS-558K/images \
 model.tokenizer.library=megatron \
 model.tokenizer.model=/path/to/tokenizer \
 model.encoder_seq_length=4096 \
 model.num_layers=24 \
 model.hidden_size=2048 \
 model.ffn_hidden_size=5440 \
 model.num_attention_heads=16 \
 model.normalization=layernorm1p \
 model.do_layer_norm_weight_decay=False \
 model.apply_query_key_layer_scaling=True \
 model.activation=fast-swiglu \
 model.headscale=False \
 model.position_embedding_type=rope \
 model.rotary_percentage=0.5 \
 model.num_query_groups=null \
 model.data.num_workers=8 \
 model.mm_cfg.llm.from_pretrained=/path/to/checkpoint \
 model.mm_cfg.llm.model_type=nvgpt \
 exp_manager.create_checkpoint_callback=True \
 model.data.conv_template=nvgpt \
 model.mm_cfg.vision_encoder.from_pretrained='openai/clip-vit-large-patch14' \
 model.mm_cfg.vision_encoder.from_hf=True \
 model.data.image_token_len=256 \
 model.optim.name="fused_adam"

## Inference

### From Pre-trained Checkpoints

If you would like to use NeVA for inference from pre-trained checkpoint via HuggingFace, you can convert from HuggingFace to `.nemo` first.

First, download the model checkpoint from HuggingFace [here](https://huggingface.co/liuhaotian/llava-v1.5-7b). The tokenizer (stored as `tokenizer.model` within the pretrained checkpoint) must be modified with the following commands:

In [ ]:
! cd /opt/sentencepiece/src/
! protoc --python_out=/opt/NeMo/scripts/tokenizers/ sentencepiece_model.proto
! python /opt/NeMo/scripts/tokenizers/add_special_tokens_to_sentencepiece.py \
--input_file /path/to/tokenizer.model \
--output_file /path/to/tokenizer_neva.model \
--is_userdefined \
--tokens "<extra_id_0>" "<extra_id_1>" "<extra_id_2>" "<extra_id_3>" \
         "<extra_id_4>" "<extra_id_5>" "<extra_id_6>" "<extra_id_7>"

Finally, convert to `.nemo` via the provided script:

In [ ]:
! python /opt/NeMo/examples/multimodal/mllm/neva/convert_hf_llava_to_neva.py \
--in-file /path/to/llava-v1.5-7b \
--out-file /path/to/llava-v1.5-7b.nemo \
--tokenizer-model /path/to/tokenizer_neva.model

### Running Inference

NeVA inference via the NeMo Framework can be quickly spun up via the NeMo Launcher and a few modifications to use the default NeVA inference config file.

Inference can be run with a similar command leveraging the provided inference script `neva_evaluation.py` within the container.

An example of an inference script execution:

In [ ]:
! torchrun --nproc_per_node=1 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_evaluation.py \
tensor_model_parallel_size=1 \
pipeline_model_parallel_size=1 \
neva_model_file=/path/to/checkpoint \
trainer.devices=1 \
trainer.precision=bf16 \
prompt_file=/path/to/prompt/file \
inference.media_base_path=/path/to/image \
output_file=path/for/output/file/ \
inference.temperature=0.2 \
inference.top_k=0 \
inference.top_p=0.9 \
inference.greedy=False \
inference.add_BOS=False \
inference.all_probs=False \
inference.repetition_penalty=1.2 \
inference.insert_media_token=null \
inference.tokens_to_generate=256 \
quantization.algorithm=awq \
quantization.enable=False

#### Running Inference via Launcher

Inference can also be run via the NeMo Launcher, where parameters are specified in the inference config file rather than CLI arguments. To customize the default config provided in `conf/config.yaml` for NeVA inference, see below.

##### Inference Config Setup
1. Modify `fw_inference` within `defaults` to use `neva/inference` 
2. In `stages`, ensure that `fw_inference` is included
3. Within the `inference.yaml` default NeVA inference config file, ensure that the path to the `prompt` file, `neva_model_file`, and `media_base_path` within `inference` are specified.

Once either the necessary checkpoints have been loaded or the training workflow is complete, inference can be executed within the launcher pipeline with the following command:

In [ ]:
! python3 main.py